# Tweaking the cellpy file format

A `cellpy` file is a `hdf5`-type file.

From v.5 it contains five top-level directories.


```python

# from cellreader.py
raw_dir = prms._cellpyfile_raw
step_dir = prms._cellpyfile_step
summary_dir = prms._cellpyfile_summary
meta_dir = "/info"  # hard-coded
fid_dir = prms._cellpyfile_fid

# from prms.py
_cellpyfile_root = "CellpyData"
_cellpyfile_raw = "/raw"
_cellpyfile_step = "/steps"
_cellpyfile_summary = "/summary"
_cellpyfile_fid = "/fid"

```


In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from pathlib import Path
from pprint import pprint

import pandas as pd

import cellpy

## Creating a fresh file from a raw-file

In [ ]:
create_cellpyfile = False
filename_full = Path(
    "/Users/jepe/cellpy_data/cellpyfiles/20181026_cen31_03_GITT_cc_01.h5"
)
filename_first = Path(
    "/Users/jepe/cellpy_data/cellpyfiles/20181026_cen31_03_GITT_cc_01_a.h5"
)
rawfile_full = Path("/Users/jepe/cellpy_data/raw/20181026_cen31_03_GITT_cc_01.res")
rawfile_full2 = Path("/Users/jepe/cellpy_data/raw/20181026_cen31_03_GITT_cc_02.res")

In [ ]:
if create_cellpyfile:
    print("--loading raw-file".ljust(50, "-"))
    c = cellpy.get(rawfile_full, mass=0.23)
    print("--saving".ljust(50, "-"))
    c.save(filename_full)
    print("--splitting".ljust(50, "-"))
    c1, c2 = c.split(4)
    c1.save(filename_first)
else:
    print("--loading cellpy-files".ljust(50, "-"))
    c1 = cellpy.get(filename_first)
    c = cellpy.get(filename_full)

## Update with loadcell

In [ ]:
cellpy.log.setup_logging(default_level="INFO")
raw_files = [rawfile_full, rawfile_full2]
# raw_files = [rawfile_full2]
cellpy_file = filename_full
c = cellpy.cellreader.CellpyData().dev_update_loadcell(raw_files, cellpy_file)

## Update with update

In [ ]:
c1 = cellpy.get(filename_first, logging_mode="INFO")
c1.dev_update(rawfile_full)

## Looking at `cellpy`´s internal parameter

In [ ]:
from cellpy import prms

In [ ]:
parent_level = prms._cellpyfile_root
raw_dir = prms._cellpyfile_raw
step_dir = prms._cellpyfile_step
summary_dir = prms._cellpyfile_summary
meta_dir = "/info"  # hard-coded
fid_dir = prms._cellpyfile_fid

In [ ]:
raw_dir

In [ ]:
parent_level + raw_dir

## Looking at a `cellpy` file using `pandas`

In [ ]:
print(f"name: {filename_full.name}")
print(f"size: {filename_full.stat().st_size/1_048_576:0.2f} Mb")

In [ ]:
with pd.HDFStore(filename_full) as store:
    pprint(store.keys())

In [ ]:
store = pd.HDFStore(filename_full)

In [ ]:
m = store.select(parent_level + meta_dir)
s = store.select(parent_level + summary_dir)
t = store.select(parent_level + step_dir)
f = store.select(parent_level + fid_dir)

In [ ]:
store.close()

In [ ]:
f.T

## Looking at a `cellpy` file using `cellpy`

In [ ]:
c = cellpy.get(filename_full)

In [ ]:
cc = c.cell

In [ ]:
fid = cc.raw_data_files[0]

In [ ]:
fid.last_data_point  # This should be used when I will implement reading only new data